# GANs models -- DCGAN

## All Required Libs and Parameters

In [32]:
# All required libs are imporeted in this cell
import os, glob, multiprocessing
import tensorflow as tf
import  numpy as np
from PIL import Image
from tensorflow.keras.layers import BatchNormalization, Conv2D, Dense, Flatten, Reshape, Conv2DTranspose, ReLU, LeakyReLU, Activation
from tensorflow.keras import Sequential, optimizers, Input
from tensorflow import keras
print(tf.__version__)

z_dim = 100
epochs = 3000000
batch_size = 128
learning_rate = 0.0002
is_training = True


2.2.0-dev20200315


## All Required Functions/Methods

In [29]:
def make_anime_dataset(img_paths, batch_size, resize=64, drop_remainder=True, shuffle=True, repeat=1):

    # @tf.function
    def _map_fn(img):
        img = tf.image.resize(img, [resize, resize])
        # img = tf.image.random_crop(img,[resize, resize])
        # img = tf.image.random_flip_left_right(img)
        # img = tf.image.random_flip_up_down(img)
        img = tf.clip_by_value(img, 0, 255)
        img = img / 127.5 - 1 #-1~1
        return img

    dataset = disk_image_batch_dataset(img_paths,
                                          batch_size,
                                          drop_remainder=drop_remainder,
                                          map_fn=_map_fn,
                                          shuffle=shuffle,
                                          repeat=repeat)
    img_shape = (resize, resize, 3)
    len_dataset = len(img_paths) // batch_size

    return dataset, img_shape, len_dataset


def batch_dataset(dataset,
                  batch_size,
                  drop_remainder=True,
                  n_prefetch_batch=1,
                  filter_fn=None,
                  map_fn=None,
                  n_map_threads=None,
                  filter_after_map=False,
                  shuffle=True,
                  shuffle_buffer_size=None,
                  repeat=None):
    # set defaults
    if n_map_threads is None:
        n_map_threads = multiprocessing.cpu_count()
    if shuffle and shuffle_buffer_size is None:
        shuffle_buffer_size = max(batch_size * 128, 2048)  # set the minimum buffer size as 2048

    # [*] it is efficient to conduct `shuffle` before `map`/`filter` because `map`/`filter` is sometimes costly
    if shuffle:
        dataset = dataset.shuffle(shuffle_buffer_size)

    if not filter_after_map:
        if filter_fn:
            dataset = dataset.filter(filter_fn)

        if map_fn:
            dataset = dataset.map(map_fn, num_parallel_calls=n_map_threads)

    else:  # [*] this is slower
        if map_fn:
            dataset = dataset.map(map_fn, num_parallel_calls=n_map_threads)

        if filter_fn:
            dataset = dataset.filter(filter_fn)

    dataset = dataset.batch(batch_size, drop_remainder=drop_remainder)

    dataset = dataset.repeat(repeat).prefetch(n_prefetch_batch)

    return dataset


def memory_data_batch_dataset(memory_data,
                              batch_size,
                              drop_remainder=True,
                              n_prefetch_batch=1,
                              filter_fn=None,
                              map_fn=None,
                              n_map_threads=None,
                              filter_after_map=False,
                              shuffle=True,
                              shuffle_buffer_size=None,
                              repeat=None):
    """Batch dataset of memory data.
    Parameters
    ----------
    memory_data : nested structure of tensors/ndarrays/lists
    """
    dataset = tf.data.Dataset.from_tensor_slices(memory_data)
    dataset = batch_dataset(dataset,
                            batch_size,
                            drop_remainder=drop_remainder,
                            n_prefetch_batch=n_prefetch_batch,
                            filter_fn=filter_fn,
                            map_fn=map_fn,
                            n_map_threads=n_map_threads,
                            filter_after_map=filter_after_map,
                            shuffle=shuffle,
                            shuffle_buffer_size=shuffle_buffer_size,
                            repeat=repeat)
    return dataset


def disk_image_batch_dataset(img_paths,
                             batch_size,
                             labels=None,
                             drop_remainder=True,
                             n_prefetch_batch=1,
                             filter_fn=None,
                             map_fn=None,
                             n_map_threads=None,
                             filter_after_map=False,
                             shuffle=True,
                             shuffle_buffer_size=None,
                             repeat=None):
    """Batch dataset of disk image for PNG and JPEG.
    Parameters
    ----------
        img_paths : 1d-tensor/ndarray/list of str
        labels : nested structure of tensors/ndarrays/lists
    """
    if labels is None:
        memory_data = img_paths
    else:
        memory_data = (img_paths, labels)

    def parse_fn(path, *label):
        img = tf.io.read_file(path)
        img = tf.image.decode_jpeg(img, channels=3)  # fix channels to 3
        return (img,) + label

    if map_fn:  # fuse `map_fn` and `parse_fn`
        def map_fn_(*args):
            return map_fn(*parse_fn(*args))
    else:
        map_fn_ = parse_fn

    dataset = memory_data_batch_dataset(memory_data,
                                        batch_size,
                                        drop_remainder=drop_remainder,
                                        n_prefetch_batch=n_prefetch_batch,
                                        filter_fn=filter_fn,
                                        map_fn=map_fn_,
                                        n_map_threads=n_map_threads,
                                        filter_after_map=filter_after_map,
                                        shuffle=shuffle,
                                        shuffle_buffer_size=shuffle_buffer_size,
                                        repeat=repeat)

    return dataset

def get_random_z(z_dim, batch_size):
    return tf.random.uniform([batch_size, z_dim], minval=-1, maxval=1)

def celoss_ones(logits):
    y = tf.ones_like(logits)
    loss = keras.losses.binary_crossentropy(y, logits, from_logits = True)
    return tf.reduce_mean(loss)

def celoss_zeros(logits):
    y = tf.zeros_like(logits)
    loss = keras.losses.binary_crossentropy(y, logits, from_logits = True)
    return tf.reduce_mean(loss)

# Loss function for Discriminator
def d_loss_fn(generator, discriminator, batch_z, batch_x, is_training):
    fake_image = generator(batch_z, is_training)
    #print(fake_image)
    d_fake_logits = discriminator(fake_image, is_training)
    d_real_logits = discriminator(batch_x, is_training)
    
    d_loss_fake = celoss_zeros(d_fake_logits)
    d_loss_real = celoss_ones(d_real_logits)
    
    return d_loss_fake + d_loss_real

def d_loss_fn2(generator, discriminator, batch_z, batch_x, is_training):
    fake_image = generator(batch_z, is_training)
    #print(fake_image)
    d_fake_logits = discriminator(fake_image, is_training)
    d_real_logits = discriminator(batch_x, is_training)
    
    d_loss_fake = celoss_zeros(d_fake_logits)
    d_loss_real = celoss_ones(d_real_logits)
    
    return d_loss_fake, d_loss_real, 0.5*d_loss_fake + 0.5*d_loss_real

def g_loss_fn(generator, discriminator, batch_z, is_training):
    fake_image = generator(batch_z, is_training)
    d_fake_logits = discriminator(fake_image, is_training)
    return celoss_ones(d_fake_logits)

def save_result(val_out, val_block_size, image_path, color_mode):
    def preprocess(img):
        img = ((img + 1.0) * 127.5).astype(np.uint8)
        # img = img.astype(np.uint8)
        return img

    preprocesed = preprocess(val_out)
    final_image = np.array([])
    single_row = np.array([])
    for b in range(val_out.shape[0]):
        # concat image into a row
        if single_row.size == 0:
            single_row = preprocesed[b, :, :, :]
        else:
            single_row = np.concatenate((single_row, preprocesed[b, :, :, :]), axis=1)

        # concat image row to final_image
        if (b+1) % val_block_size == 0:
            if final_image.size == 0:
                final_image = single_row
            else:
                final_image = np.concatenate((final_image, single_row), axis=0)

            # reset single row
            single_row = np.array([])

    if final_image.shape[2] == 1:
        final_image = np.squeeze(final_image, axis=2)
    #toimage(final_image).save(image_path)
    #print(image_path)
    Image.fromarray(final_image).save(image_path)
    
img_path = glob.glob('./faces/*.jpg')
print("Num of Files:",len(img_path))

dataset, img_shape, _ = make_anime_dataset(img_path, batch_size, resize = 64)
print(dataset, img_shape)
sample = next(iter(dataset))
print(sample.shape, tf.reduce_max(sample).numpy(), tf.reduce_min(sample).numpy())
dataset = dataset.repeat(100)
db_iter = iter(dataset)

Num of Files: 51223
<PrefetchDataset shapes: (128, 64, 64, 3), types: tf.float32> (64, 64, 3)
(128, 64, 64, 3) 1.0 -1.0


## Model Definintions and Test Code

### SubClass Mode

In [40]:
class Generator(keras.Model):
    def __init__(self):
        super(Generator, self).__init__()
        self.s = 2
        self.k = 4
        self.n_f = 1024
        
        self.dense1 = Dense(self.s * self.s * self.n_f)
        self.reshape1 = Reshape(target_shape = (self.s, self.s, self.n_f))
        self.conv1 = Conv2DTranspose(512, kernel_size = self.k, strides = 2, padding = 'same')
        self.bn1 = BatchNormalization()
        self.conv2 = Conv2DTranspose(256, kernel_size = self.k, strides = 2, padding = 'same')
        self.bn2 = BatchNormalization()
        self.conv3 = Conv2DTranspose(128, kernel_size = self.k, strides = 2, padding = 'same')
        self.bn3 = BatchNormalization()
        self.conv4 = Conv2DTranspose(64, kernel_size = self.k, strides = 2, padding = 'same')
        self.bn4 = BatchNormalization()
        self.conv5 = Conv2DTranspose(3, kernel_size = self.k, strides = 2, padding = 'same')
    def call(self, inputs, training = None):
        x = inputs # inputs [batch, z_dim]
        x = self.dense1(x)
        x = tf.nn.leaky_relu(x)
        x = self.reshape1(x)
        
        x = tf.nn.leaky_relu(self.bn1(self.conv1(x), training = training))
        x = tf.nn.leaky_relu(self.bn2(self.conv2(x), training = training))
        x = tf.nn.leaky_relu(self.bn3(self.conv3(x), training = training))
        x = tf.nn.leaky_relu(self.bn4(self.conv4(x), training = training))
        # No BatchNormalization for output layer of Generator
        x = tf.tanh(self.conv5(x))
        
        return x
class Discriminator(keras.Model):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.s = 4
        self.k = 4
        #self.n_f = 1024
        
        self.conv1 = Conv2D(64, kernel_size = self.k, strides = 2, padding = 'same')
        self.conv2 = Conv2D(128, kernel_size = self.k, strides = 2, padding = 'same')
        self.bn2 = BatchNormalization()
        self.conv3 = Conv2D(256, kernel_size = self.k, strides = 2, padding = 'same')
        self.bn3 = BatchNormalization()
        self.conv4 = Conv2D(512, kernel_size = self.k, strides = 2, padding = 'same')
        self.bn4 = BatchNormalization()
        self.flatten = Flatten()
        self.dense = Dense(1)
        
    def call(self, inputs, training = None):
        x = inputs
        x = tf.nn.leaky_relu(self.conv1(x))
        x = tf.nn.leaky_relu(self.bn2(self.conv2(x), training = training))
        x = tf.nn.leaky_relu(self.bn3(self.conv3(x), training = training))
        x = tf.nn.leaky_relu(self.bn4(self.conv4(x), training = training))
        x = self.dense(self.flatten(x))
        return x
        
        # No BatchNormalization for input layer of Discriminator
        
def main():
    generator = Generator()
    generator.build(input_shape=(None, z_dim))
    #generator.summary()
    
    discriminator = Discriminator()
    discriminator.build(input_shape=(None, 64, 64, 3))
    #discriminator.summary()
    
    g_opt = optimizers.Adam(learning_rate = learning_rate, beta_1=0.5)
    d_opt = optimizers.Adam(learning_rate = learning_rate, beta_1=0.5)
    
    d_losses, g_losses = [], []
    
    for epoch in range(epochs):
        for _ in range(1):
            batch_z = get_random_z(z_dim, batch_size)
            batch_x = next(db_iter)
            
            with tf.GradientTape() as d_tape:
                d_loss = d_loss_fn(generator, discriminator, batch_z, batch_x, is_training)
            d_grads = d_tape.gradient(d_loss, discriminator.trainable_variables)
            d_opt.apply_gradients(zip(d_grads, discriminator.trainable_variables))
        
        batch_zz = get_random_z(z_dim, batch_size)
        batch_xx = next(db_iter)
        
        with tf.GradientTape() as g_tape:
            g_loss = g_loss_fn(generator, discriminator, batch_zz, is_training)
        g_grads = g_tape.gradient(g_loss, generator.trainable_variables)
        g_opt.apply_gradients(zip(g_grads, generator.trainable_variables))
            
        if epoch % 100 == 0:
            print("Epoch:", epoch, "D-loss:", float(d_loss), "G-loss:", float(g_loss))
            z = tf.random.uniform([100, z_dim], minval=-1, maxval=1)
            generated_images = generator(z, training = False)
            img_path = os.path.join('gan_images', 'gan-%d.png'%epoch)
            save_result(generated_images.numpy(), 10, img_path, color_mode='P')

            d_losses.append(float(d_loss))
            g_losses.append(float(g_loss))

        if epoch % 10000 == 1:
            generator.save_weights('generator.ckpt')
            discriminator.save_weights('discriminator.ckpt')
if __name__ == "__main__":
    main()

(None, 32, 32, 64)
(None, 16, 16, 128)
(None, 8, 8, 256)
(None, 4, 4, 512)
(None, 1)
(128, 32, 32, 64)
(128, 16, 16, 128)
(128, 8, 8, 256)
(128, 4, 4, 512)
(128, 1)
(128, 32, 32, 64)
(128, 16, 16, 128)
(128, 8, 8, 256)
(128, 4, 4, 512)
(128, 1)
(128, 32, 32, 64)
(128, 16, 16, 128)
(128, 8, 8, 256)
(128, 4, 4, 512)
(128, 1)
Epoch: 0 D-loss: 1.4889330863952637 G-loss: 6.032578468322754
(128, 32, 32, 64)
(128, 16, 16, 128)
(128, 8, 8, 256)
(128, 4, 4, 512)
(128, 1)
(128, 32, 32, 64)
(128, 16, 16, 128)
(128, 8, 8, 256)
(128, 4, 4, 512)
(128, 1)
(128, 32, 32, 64)
(128, 16, 16, 128)
(128, 8, 8, 256)
(128, 4, 4, 512)
(128, 1)
(128, 32, 32, 64)
(128, 16, 16, 128)
(128, 8, 8, 256)
(128, 4, 4, 512)
(128, 1)
(128, 32, 32, 64)
(128, 16, 16, 128)
(128, 8, 8, 256)
(128, 4, 4, 512)
(128, 1)
(128, 32, 32, 64)
(128, 16, 16, 128)
(128, 8, 8, 256)
(128, 4, 4, 512)
(128, 1)
(128, 32, 32, 64)
(128, 16, 16, 128)
(128, 8, 8, 256)
(128, 4, 4, 512)
(128, 1)
(128, 32, 32, 64)
(128, 16, 16, 128)
(128, 8, 8, 256)

(128, 32, 32, 64)
(128, 16, 16, 128)
(128, 8, 8, 256)
(128, 4, 4, 512)
(128, 1)
(128, 32, 32, 64)
(128, 16, 16, 128)
(128, 8, 8, 256)
(128, 4, 4, 512)
(128, 1)
(128, 32, 32, 64)
(128, 16, 16, 128)
(128, 8, 8, 256)
(128, 4, 4, 512)
(128, 1)
(128, 32, 32, 64)
(128, 16, 16, 128)
(128, 8, 8, 256)
(128, 4, 4, 512)
(128, 1)
(128, 32, 32, 64)
(128, 16, 16, 128)
(128, 8, 8, 256)
(128, 4, 4, 512)
(128, 1)
(128, 32, 32, 64)
(128, 16, 16, 128)
(128, 8, 8, 256)
(128, 4, 4, 512)
(128, 1)
(128, 32, 32, 64)
(128, 16, 16, 128)
(128, 8, 8, 256)
(128, 4, 4, 512)
(128, 1)
(128, 32, 32, 64)
(128, 16, 16, 128)
(128, 8, 8, 256)
(128, 4, 4, 512)
(128, 1)
(128, 32, 32, 64)
(128, 16, 16, 128)
(128, 8, 8, 256)
(128, 4, 4, 512)
(128, 1)
(128, 32, 32, 64)
(128, 16, 16, 128)
(128, 8, 8, 256)
(128, 4, 4, 512)
(128, 1)
(128, 32, 32, 64)
(128, 16, 16, 128)
(128, 8, 8, 256)
(128, 4, 4, 512)
(128, 1)
(128, 32, 32, 64)
(128, 16, 16, 128)
(128, 8, 8, 256)
(128, 4, 4, 512)
(128, 1)
(128, 32, 32, 64)
(128, 16, 16, 128)
(12

KeyboardInterrupt: 

### Functional API Mode

In [47]:
# Subclass needs to specify input when calling .build(None,...)
# API needs to specify input shape without batch dim.
def build_generator_model(z_dim = 100, batch_size = 128, s = 2 , k = 4 , n_filters = 1024):    
    noise = Input(shape = [z_dim])
    x = Dense(s*s*n_filters)(noise)
    x = LeakyReLU()(x)
    x = Reshape(target_shape=[s, s, n_filters])(x)
    #print(x.shape)
    
    # Conv 1
    x = Conv2DTranspose(filters = 512, kernel_size = k, strides = 2, padding ='same')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU()(x)
    #print(x.shape)
    
    # Conv 2
    x = Conv2DTranspose(filters = 256, kernel_size = k, strides = 2, padding ='same')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU()(x)
    #print(x.shape)
    
    # Conv 3
    x = Conv2DTranspose(filters = 128, kernel_size = k, strides = 2, padding ='same')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU()(x)
    #print(x.shape)
    
    # Conv 4
    x = Conv2DTranspose(filters = 64, kernel_size = k, strides = 2, padding ='same')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU()(x)
    #print(x.shape)
    
    # Conv 5
    x = Conv2DTranspose(filters = 3, kernel_size = k, strides = 2, padding = 'same')(x)
    logits = Activation('tanh', name='logits')(x)
    #print(x.shape)
    
    return tf.keras.Model(noise, logits, name='Generator')

def build_discriminator_model(z_dim=100, k = 4):
    image_input = Input(shape=[64, 64, 3])
    x = Conv2D(filters = 64, kernel_size = k, strides = 2, padding = 'same')(image_input)
    x = LeakyReLU()(x)
    #print(x.shape)
    
    x = Conv2D(filters = 128, kernel_size = k, strides = 2, padding = 'same')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU()(x)
    #print(x.shape)
    
    x = Conv2D(filters = 256, kernel_size = k, strides = 2, padding = 'same')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU()(x)
    #print(x.shape)
    
    x = Conv2D(filters = 512, kernel_size = k, strides = 2, padding = 'same')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU()(x)
    #print(x.shape)
    
    x = Flatten()(x)
    logits = Dense(1)(x)
    #print(logits.shape)
    return tf.keras.Model(image_input, logits, name='Discriminator')

def main():
    gen_model = build_generator_model()
    dis_model = build_discriminator_model()
    
    g_opt = optimizers.Adam(learning_rate = learning_rate, beta_1=0.5)
    d_opt = optimizers.Adam(learning_rate = learning_rate, beta_1=0.5)
    
    d_losses, g_losses = [], []
    
    for epoch in range(epochs):
        for _ in range(1):
            batch_z = get_random_z(z_dim, batch_size)
            batch_x = next(db_iter)
            
            with tf.GradientTape() as d_tape:
                d_loss = d_loss_fn(gen_model, dis_model, batch_z, batch_x, is_training)
            d_grads = d_tape.gradient(d_loss, dis_model.trainable_variables)
            d_opt.apply_gradients(zip(d_grads, dis_model.trainable_variables))
        
        batch_zz = get_random_z(z_dim, batch_size)
        batch_xx = next(db_iter)

        with tf.GradientTape() as g_tape:
            g_loss = g_loss_fn(gen_model, dis_model, batch_zz, is_training)
        g_grads = g_tape.gradient(g_loss, gen_model.trainable_variables)
        g_opt.apply_gradients(zip(g_grads, gen_model.trainable_variables))

        if epoch % 100 == 0:
            print("Epoch:", epoch, "D-loss:", float(d_loss), "G-loss:", float(g_loss))
            z = tf.random.uniform([100, z_dim], minval=-1, maxval=1)
            generated_images = gen_model(z, training = False)
            img_path = os.path.join('gan_images', 'api_gan-%d.png'%epoch)
            save_result(generated_images.numpy(), 10, img_path, color_mode='P')

            d_losses.append(float(d_loss))
            g_losses.append(float(g_loss))

        if epoch % 10000 == 1:
            gen_model.save_weights('generator.ckpt')
            dis_model.save_weights('discriminator.ckpt')
if __name__ == "__main__":
    main()
        


Epoch: 0 D-loss: 1.5948282480239868 G-loss: 5.737722396850586
Epoch: 100 D-loss: 1.0108578205108643 G-loss: 2.7868332862854004
Epoch: 200 D-loss: 1.5291911363601685 G-loss: 4.22914981842041
Epoch: 300 D-loss: 0.8474067449569702 G-loss: 3.167870044708252
Epoch: 400 D-loss: 0.9078989624977112 G-loss: 5.156543254852295
Epoch: 500 D-loss: 1.0400261878967285 G-loss: 2.2575838565826416
Epoch: 600 D-loss: 0.8453784584999084 G-loss: 4.009437084197998
Epoch: 700 D-loss: 0.6565330028533936 G-loss: 3.2830617427825928
Epoch: 800 D-loss: 0.8159675002098083 G-loss: 4.475857734680176
Epoch: 900 D-loss: 0.5756043195724487 G-loss: 2.435945510864258
Epoch: 1000 D-loss: 1.4895097017288208 G-loss: 2.402553081512451
Epoch: 1100 D-loss: 0.6468749642372131 G-loss: 3.6792359352111816
Epoch: 1200 D-loss: 1.1983184814453125 G-loss: 3.726593255996704
Epoch: 1300 D-loss: 0.9159888625144958 G-loss: 4.501795768737793
Epoch: 1400 D-loss: 1.3910218477249146 G-loss: 1.8285226821899414
Epoch: 1500 D-loss: 0.90816950798

Epoch: 12600 D-loss: 0.15633945167064667 G-loss: 7.504440784454346
Epoch: 12700 D-loss: 0.049245819449424744 G-loss: 8.571148872375488
Epoch: 12800 D-loss: 0.33248794078826904 G-loss: 5.765406608581543
Epoch: 12900 D-loss: 0.018209179863333702 G-loss: 7.256743431091309
Epoch: 13000 D-loss: 0.09191304445266724 G-loss: 4.788175582885742
Epoch: 13100 D-loss: 0.10258734971284866 G-loss: 6.195735931396484
Epoch: 13200 D-loss: 0.14156274497509003 G-loss: 5.407291889190674
Epoch: 13300 D-loss: 0.21264532208442688 G-loss: 7.441789150238037
Epoch: 13400 D-loss: 0.19592316448688507 G-loss: 8.862916946411133
Epoch: 13500 D-loss: 0.17427287995815277 G-loss: 8.341344833374023
Epoch: 13600 D-loss: 0.1856236755847931 G-loss: 5.131638526916504
Epoch: 13700 D-loss: 0.41536784172058105 G-loss: 12.447162628173828
Epoch: 13800 D-loss: 0.04603038728237152 G-loss: 4.747023582458496
Epoch: 13900 D-loss: 0.11698566377162933 G-loss: 6.324587821960449
Epoch: 14000 D-loss: 3.7645351886749268 G-loss: 8.4994525909

StopIteration: 

### Sequential Mode

In [ ]:
# DCGAN implementation with Functional API mode
